In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import FuncFormatter

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment
from defdap.quat import Quat

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks
from scipy.linalg import sqrtm, polar




import os

# get dictools stuff 
import sys
# sys.path.append("c:/work/hrdic-tools/")
# import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

In [2]:
exp = experiment.Experiment()

# load DIC data 
data_dir = Path('.')
dic_frame = experiment.Frame()
for dic_file in sorted(data_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)

Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)


In [3]:
ebsd_frame = experiment.Frame()
ebsd.Map(data_dir / 'Post_EBSD/map_small.cpr',
         increment=exp.increments[0], frame=ebsd_frame)

Loaded EBSD data (dimensions: 1825 x 1750 pixels, step size: 0.2 um)


In [4]:
ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.set_homog_point()

dic_map = exp.increments[0].maps['hrdic']
# dic_map.set_homog_point(vmin=0,vmax=0.1)

In [5]:
# dic_map = exp.increments[0].maps['hrdic']
# dic_frame.homog_points = [(952, 825),
#  (1506, 586),
#  (1597, 69),
#  (858, 167),
#  (1143, 700),
#  (1205, 191),
#  (1769, 553),
#  (1273, 1097),
#  (1539, 1171),
#  (1761, 1666),
#  (1109, 1381),
#  (1565, 1618),
#  (82, 358),
#  (304, 563),
#  (635, 671),
#  (923, 538),
#  (927, 1116),
#  (235, 1302),
#  (871, 1446),
#  (96, 1743)]

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_frame.homog_points = [(1946, 1565),
#  (2375, 1384),
#  (2443, 999),
#  (1877, 1079),
#  (2094, 1469),
#  (2142, 1091),
#  (2574, 1359),
#  (2193, 1760),
#  (2396, 1823),
#  (2571, 2196),
#  (2065, 1989),
#  (2418, 2156),
#  (1279, 1218),
#  (1454, 1369),
#  (1700, 1457),
#  (1925, 1345),
#  (1929, 1788),
#  (1391, 1934),
#  (1881, 2037),
#  (1281, 2274)]

In [6]:
ebsd_map = exp.increments[0].maps['ebsd']
ebsd_frame.homog_points = [(274, 381),
 (891, 265),
 (1613, 265),
 (958, 760),
 (1148, 1241),
 (1425, 1035),
 (378, 1109),
 (390, 1409),
 (1615, 1407),
 (889, 1245),
 (710, 635),
 (576, 558),
 (556, 696),
 (318, 703),
 (545, 499),
 (1216, 958),
 (1375, 752),
 (781, 976),
 (1303, 732),
 (1061, 746),
 (1514, 594),
 (1274, 510),
 (1435, 427),
 (1117, 664),
 (425, 1075),
 (631, 1400),
 (631, 1036),
 (769, 1126),
 (1152, 942),
 (1240, 1339),
 (1528, 1149),
 (1084, 1193)]


dic_frame.homog_points = [(83, 363),
 (856, 170),
 (1759, 146),
 (949, 829),
 (1186, 1442),
 (1536, 1173),
 (233, 1304),
 (245, 1687),
 (1761, 1668),
 (868, 1444),
 (634, 672),
 (465, 576),
 (444, 757),
 (144, 785),
 (425, 495),
 (1273, 1087),
 (1470, 805),
 (727, 1117),
 (1382, 780),
 (1074, 814),
 (1643, 593),
 (1341, 480),
 (1543, 362),
 (1147, 700),
 (290, 1256),
 (548, 1663),
 (545, 1195),
 (719, 1310),
 (1194, 1069),
 (1303, 1594),
 (1654, 1333),
 (1109, 1382)]

In [7]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    



    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs='line'
    )
    plt.tight_layout()

Finished building quaternion array (0:00:08) 
Finished finding grain boundaries (0:00:21) 


In [8]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.plot_map(
        'r_ang'
    )

In [9]:
# find special boundaries 
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)


# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# calculate neighbour network
ebsd_map.build_neighbour_network()

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
# store all gbs pairs with misorientation for later us 

all_gb_info = []
twin_gb_info = []
twin_lines = []

for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False

    # calculate grain ref orientation
    grain1.calc_average_ori()
    grain2.calc_average_ori()

    misori = grain2.ref_ori * grain1.ref_ori.conjugate

    # # calculate misorientation angle in degrees 
    misori_ang = 2*np.arccos(grain1.ref_ori.mis_ori(grain2.ref_ori,ebsd_map.crystal_sym))*180/np.pi

    # add to list of everything 
    all_gb_info.append([grain1,grain2,misori,misori_ang,b_seg])

    # check if twin and add to twin list
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines.append(b_seg)
    twin_gb_info.append([grain1,grain2,misori,misori_ang,b_seg])




# make into boundary set
s_bounds = ebsd.BoundarySet.from_boundary_segments(twin_lines)

Finished finding grains (0:00:05) twork..
Finished constructing neighbour network (0:00:07) 


In [10]:
# plot misorientation distribution function 

# sanity check for all twin boundaries having a MO ~ 60 degrees 
twin_mo_angle = []
for i in range(len(twin_gb_info)):
    twin_mo_angle.append(twin_gb_info[i][3])



all_mo_angle = []
for i in range(len(all_gb_info)):
    all_mo_angle.append(all_gb_info[i][3])

fig,ax = plt.subplots()
ax.hist(all_mo_angle,range=(0,65),bins=50,density=True)
ax.set_ylabel('Probability density / -')
ax.set_xlabel('Misorientation angle / °')
plt.tight_layout()

Mask individual boundaries to extract max shear for each

In [11]:
# functions

def mask_and_extract(dic_map,mask):
    # masks DIC map data BUT not in the DIC map object
    # extracts the data first, then masks, then returns the data 

    # max shear
    ems = copy.deepcopy(dic_map.data['max_shear'])
    ems[mask] = np.nan

    # strain components 
    e = copy.deepcopy(dic_map.data['e'])
    e[:,:,mask] = np.nan

    # rotations 
    r_ang = copy.deepcopy(dic_map.data['r_ang'])
    r_ang[mask] = np.nan

    return ems, e, r_ang

def make_log_log_histogram(fig,ax,data_list,bin_range,bins,density,colour_list):
    bin_list = np.logspace(np.log10(bin_range[0]),np.log10(bin_range[-1]),bins)

    for i in range(0,len(data_list)):

        hist = np.histogram(data_list[i].flatten(), bins=bin_list, density=density)

        y_vals = hist[0]
        x_vals = 0.5 * (hist[1][1:] + hist[1][:-1])

        ax.plot(x_vals,y_vals,'.',mfc=colour_list[i],mec=colour_list[i],ms=5)

    return  fig, ax 

In [71]:
# loop over every gb in dic map, extract misorientation, Ems, E, r angle 

dilation = 5

dic_gb_mori     = []
dic_gb_ems      = []
dic_gb_e        = []
dic_gb_r_ang    = []


for i in range(len(all_gb_info)):
    this_b_seg = ebsd.BoundarySet.from_boundary_segments([all_gb_info[i][4]])
    this_b_seg_dic = hrdic.BoundarySet.from_ebsd_boundaries(dic_map,this_b_seg)

    # check if this gb is in the DIC ROI 
    if np.all(this_b_seg_dic.image) == False: 
        dic_map.generate_threshold_mask(this_b_seg_dic.image,dilation=dilation,preview=False)

        # # this is very slow and makes a copy of each array many times, but keeps the spatial relationshp of the grain boundary
        # this_mask = ~dic_map.mask
        # this_ems, this_e, this_r_ang = mask_and_extract(dic_map,this_mask)

        this_ems = dic_map.data['max_shear'][dic_map.mask]
        this_r_ang = dic_map.data['r_ang'][dic_map.mask]

        dic_gb_mori.append(all_gb_info[i][3])
        dic_gb_ems.append(this_ems)
        # dic_gb_e.append(this_e)
        dic_gb_r_ang.append(this_r_ang)

Filtering will remove 234 \ 3996001 (0.006 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 540 \ 3996001 (0.014 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 182 \ 3996001 (0.005 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 259 \ 3996001 (0.006 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Filtering will remove 171 \ 3996001 (0.004 %) datapoints in map
Filtering will remove 0 \ 3236401 (0.000 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data


In [332]:
dic_gb_r_ang_mag = []
for i in range(len(dic_gb_r_ang)):
    dic_gb_r_ang_mag.append(np.abs(dic_gb_r_ang[i]))

In [457]:
# make every point separate for all segemented points such that we have 
# point 1 in gb 1, mo_ang, r_ang, ems
# point 2 in gb 1, mo_ang, r_ang, ems
# .....   .. ..    ......  .....  ... 
# point n in gb 1, mo_ang, r_ang, ems
# point 1 in gb 2, mo_ang, r_ang, ems
# point 2 in gb 2, mo_ang, r_ang, ems

dic_gb_ems_all = []
dic_gb_r_ang_all = []
dic_gb_mis_ori_all = []

for i in range(len(dic_gb_ems)):
    # get this misorientation 
    mo_i = dic_gb_mori[i]

    for j in range(len(dic_gb_ems[i])):
        dic_gb_mis_ori_all.append(mo_i)
        dic_gb_ems_all.append(dic_gb_ems[i][j])
        dic_gb_r_ang_all.append(dic_gb_r_ang[i][j])
    
dic_gb_ems_all = np.asarray(dic_gb_ems_all)
dic_gb_r_ang_all = np.asarray(dic_gb_r_ang_all)
dic_gb_mis_ori_all = np.asarray(dic_gb_mis_ori_all)

In [544]:
def scatter_with_marg_hists(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range,var1_highlight = False,percentile=75):
    # scatter around edge of 2d histogram

    # Create a Figure, which doesn't have to be square.
    fig = plt.figure()

    # Create the main Axes.
    ax = fig.add_subplot()

    # The main Axes' aspect can be fixed.
    # ax.set_aspect('')
    # Create marginal Axes, which have 25% of the size of the main Axes.  Note that
    # the inset Axes are positioned *outside* (on the right and the top) of the
    # main Axes, by specifying axes coordinates greater than 1.  Axes coordinates
    # less than 0 would likewise specify positions on the left and the bottom of
    # the main Axes.
    ax_histx = ax.inset_axes([0, 1.1, 1, 0.25], sharex=ax)
    ax_histy = ax.inset_axes([1.1, 0, 0.25, 1], sharey=ax)
    ax_leg   = ax.inset_axes([1.1,1.1,0.25,0.25])
    ax_leg.axis('off')


    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # sort variable 2 by variable 1 in increasing order 
    var1,var2 = zip(*sorted(zip(var1,var2)))

    var1 = np.asarray(var1)
    var2 = np.asarray(var2)

    # colours
    c_normal = 'grey'
    c_highlight = 'lightsalmon'

    # need to make the histograms first as they control the binning for the curves in the main plot
    h = ax_histx.hist(var1, range=var1_range, bins=var1_bins,density=True,color=c_normal,edgecolor='dimgrey',linewidth=1.2,alpha=0.5)
    ax_histy.hist(var2, range=var2_range, bins=var2_bins, orientation='horizontal',density=True,color=c_normal,edgecolor='dimgrey',linewidth=1.2,alpha=0.5)

    if var1_highlight != False: 
        c_list = []
        var1_for_highlight = []
        var2_for_highlight = []
        for i,v in enumerate(var1):
            if (v > var1_highlight[0]) & (v < var1_highlight[1]):
                c_list.append(c_highlight)
                var1_for_highlight.append(v)
                var2_for_highlight.append(var2[i])
            else:
                c_list.append(c_normal)


        var2_for_highlight = np.asarray(var2_for_highlight)
        # ax_histy.hist(var2_for_highlight, range=var2_range, bins=var2_bins, orientation='horizontal',density=False,color=c_highlight,edgecolor='dimgrey',linewidth=1.2,alpha=0.5)
        xlims = ax_histy.get_xbound()
        hline_hlight = ax_histy.hlines(np.nanpercentile(var2_for_highlight,percentile),xmin=xlims[0],xmax=xlims[-1],colors=c_highlight,label='$\Sigma$3 boundaries',alpha=0.5)
        hline_all = ax_histy.hlines(np.nanpercentile(var2,percentile),xmin=xlims[0],xmax=xlims[-1],colors=c_normal,label='All boundaries',alpha=0.5)
    else:
        c_list = c_normal


    # the scatter plot:
    ax.scatter(var1, var2,marker='.',c=c_list,alpha=0.05)
    ax.set_xlim(var1_range)
    ax.set_ylim(var2_range)

    # 2d histogram
    # h = ax.hist2d(var1,
    #               var2,
    #               bins=[var1_bins,var2_bins],
    #               range=[var1_range,var2_range],density=True,
    #               cmap = cmap,
    #             #   norm = mpl.colors.LogNorm()
    #     )


    # add lines for median and standard deviation 
    var2_med = []
    var2_std = []
    var2_pclo = []
    var2_pchi = []

    for i in range(1,len(h[1])):
        var2_i =  np.where((var1 > h[1][i-1]) & (var1 <h[1][i]),var2,np.nan)

        var2_i = var2_i[~np.isnan(var2_i)]


        var2_med.append(np.median(var2_i))
        var2_std.append(np.std(var2_i))
        try:
            var2_pclo.append(np.percentile(var2_i,100 - percentile))
        except:
            var2_pclo.append(np.nan)
        try:
            var2_pchi.append(np.percentile(var2_i,percentile))
        except:
            var2_pchi.append(np.nan)


    var2_med = np.asarray(var2_med)
    var2_std = np.asarray(var2_std)

    bin_centres = (h[1][0:-1]+h[1][1:])*0.5

    line_colour = 'deepskyblue'

    ax.plot(bin_centres,var2_med,'-',color=line_colour)
    # ax.plot(h[1][0:-1],var2_med-var2_std,'--',color=line_colour)
    # ax.plot(h[1][0:-1],var2_med+var2_std,'--',color=line_colour)
    ax.plot(bin_centres,var2_pclo,'--',color=line_colour)
    ax.plot(bin_centres,var2_pchi,'--',color=line_colour)
    # ax.fill_between(h[1][0:-1],var2_med-var2_std,var2_med+var2_std,color='fuchsia',alpha=0.5)



    # labelling
    ax.set_xlabel(var1_label)
    ax.set_ylabel(var2_label)
    ax_histx.set_ylabel('Prob. dens. / -')
    # ax_leg.legend(handles=[hline_all,hline_hlight])

    ax_histy.annotate('$\Sigma$3 GBs',(xlims[-1],np.nanpercentile(var2_for_highlight,percentile)),va='center',textcoords='offset points',xytext=(10,0))
    ax_histy.annotate('All GBs',(xlims[-1],np.nanpercentile(var2,percentile)),va='center',textcoords='offset points',xytext=(10,0))

    # cbar = fig.colorbar(h[3],ax=ax,location='bottom',label='Probability density')
    # cbar.set_ticks([1e-2,5e-2,1e-1],minor=True)
    plt.tight_layout()

    return fig,ax

In [486]:
def scatter_with_marg_histx_violiny(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range,var1_highlight = False,percentile=75):
    # scatter around edge of 2d histogram

    # Create a Figure, which doesn't have to be square.
    fig = plt.figure()

    # Create the main Axes.
    ax = fig.add_subplot()

    # The main Axes' aspect can be fixed.
    # ax.set_aspect('')
    # Create marginal Axes, which have 25% of the size of the main Axes.  Note that
    # the inset Axes are positioned *outside* (on the right and the top) of the
    # main Axes, by specifying axes coordinates greater than 1.  Axes coordinates
    # less than 0 would likewise specify positions on the left and the bottom of
    # the main Axes.
    ax_histx = ax.inset_axes([0, 1.05, 1, 0.25], sharex=ax)
    ax_histy = ax.inset_axes([1.05, 0, 0.25, 1], sharey=ax)


    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # sort variable 2 by variable 1 in increasing order 
    var1,var2 = zip(*sorted(zip(var1,var2)))

    var1 = np.asarray(var1)
    var2 = np.asarray(var2)

    # colours
    c_normal = 'grey'
    c_highlight = 'lightsalmon'

    # need to make the histograms first as they control the binning for the curves in the main plot
    h = ax_histx.hist(var1, range=var1_range, bins=var1_bins,density=True,color=c_normal,edgecolor='dimgrey',linewidth=1.2,alpha=0.5)
    # ax_histy.hist(var2, range=var2_range, bins=var2_bins, orientation='horizontal',density=False,color=c_normal,edgecolor='dimgrey',linewidth=1.2,alpha=0.5)

    if var1_highlight != False: 
        c_list = []
        var1_for_highlight = []
        var2_for_highlight = []
        var2_for_unhighlight = []
        for i,v in enumerate(var1):
            if (v > var1_highlight[0]) & (v < var1_highlight[1]):
                c_list.append(c_highlight)
                var1_for_highlight.append(v)
                var2_for_highlight.append(var2[i])
            else:
                var2_for_unhighlight.append(var2[i])
                c_list.append(c_normal)


        var2_for_highlight = np.asarray(var2_for_highlight)
        var2_for_unhighlight = np.asarray(var2_for_unhighlight)
        ax_histy.violinplot([var2_for_unhighlight,var2_for_highlight],positions=[1,2],showextrema=False,showmeans=False,showmedians=True)
    else:
        c_list = c_normal
        ax_histy.violinplot(var2,showextrema=False,showmeans=False,showmedians=True)


    # the scatter plot:
    ax.scatter(var1, var2,marker='.',c=c_list,alpha=0.05)
    ax.set_xlim(var1_range)
    ax.set_ylim(var2_range)

    # 2d histogram
    # h = ax.hist2d(var1,
    #               var2,
    #               bins=[var1_bins,var2_bins],
    #               range=[var1_range,var2_range],density=True,
    #               cmap = cmap,
    #             #   norm = mpl.colors.LogNorm()
    #     )


    # add lines for median and standard deviation 
    var2_med = []
    var2_std = []
    var2_pclo = []
    var2_pchi = []

    for i in range(1,len(h[1])):
        var2_i =  np.where((var1 > h[1][i-1]) & (var1 <h[1][i]),var2,np.nan)

        var2_i = var2_i[~np.isnan(var2_i)]


        var2_med.append(np.median(var2_i))
        var2_std.append(np.std(var2_i))
        try:
            var2_pclo.append(np.percentile(var2_i,100 - percentile))
        except:
            var2_pclo.append(np.nan)
        try:
            var2_pchi.append(np.percentile(var2_i,percentile))
        except:
            var2_pchi.append(np.nan)


    var2_med = np.asarray(var2_med)
    var2_std = np.asarray(var2_std)

    bin_centres = (h[1][0:-1]+h[1][1:])*0.5

    line_colour = 'deepskyblue'

    ax.plot(bin_centres,var2_med,'-',color=line_colour)
    # ax.plot(h[1][0:-1],var2_med-var2_std,'--',color=line_colour)
    # ax.plot(h[1][0:-1],var2_med+var2_std,'--',color=line_colour)
    ax.plot(bin_centres,var2_pclo,'--',color=line_colour)
    ax.plot(bin_centres,var2_pchi,'--',color=line_colour)
    # ax.fill_between(h[1][0:-1],var2_med-var2_std,var2_med+var2_std,color='fuchsia',alpha=0.5)



    # labelling
    ax.set_xlabel(var1_label)
    ax.set_ylabel(var2_label)
    ax_histx.set_ylabel('Prob. dens. / -')

    # cbar = fig.colorbar(h[3],ax=ax,location='bottom',label='Probability density')
    # cbar.set_ticks([1e-2,5e-2,1e-1],minor=True)
    plt.tight_layout()

    return fig,ax

In [545]:
var1 = dic_gb_mis_ori_all
var2 = dic_gb_ems_all

var1_highlight = [57.5,62.5] 

var1_label = 'Grain boundary misorientation / ° '
var2_label = '$E_{\mathrm{eff}}$/ - '

var1_range = [10,65]
var1_bins = 22

var2_range = [0,0.5]
var2_bins = 100

scatter_with_marg_hists(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range,var1_highlight=var1_highlight,percentile=99)

c:\Users\bepoole\AppData\Local\miniforge3\envs\stitchndic\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\bepoole\AppData\Local\miniforge3\envs\stitchndic\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\bepoole\AppData\Local\miniforge3\envs\stitchndic\lib\site-packages\numpy\_core\_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\bepoole\AppData\Local\miniforge3\envs\stitchndic\lib\site-packages\numpy\_core\_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\bepoole\AppData\Local\miniforge3\envs\stitchndic\lib\site-packages\numpy\_core\_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.

(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Grain boundary misorientation / ° ', ylabel='$E_{\\mathrm{eff}}$/ - '>)

In [ ]:
var1 = dic_gb_mis_ori_all
var2 = np.abs(dic_gb_r_ang_all)

var1_highlight = [57.5,62.5] 

var1_label = 'Grain boundary misorientation / ° '
var2_label = 'In-plane rotation magnitude/ ° '

var1_range = [10,65]
var1_bins = 22

var2_range = [0,20]
var2_bins = 100

scatter_with_marg_hists(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range,var1_highlight=var1_highlight,percentile=99)

(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Grain boundary misorientation / ° ', ylabel='In-plane rotation magnitude/ ° '>)

In [336]:
# means 
dic_gb_ems_mean = []
dic_gb_r_ang_mean = []
dic_gb_r_ang_mag_mean = []
for i in range(len(dic_gb_ems)):
    dic_gb_ems_mean.append(np.nanmean(dic_gb_ems[i]))
    dic_gb_r_ang_mean.append(np.nanmean(dic_gb_r_ang_mag[i]))
    dic_gb_r_ang_mag_mean.append(np.nanmean(dic_gb_r_ang_mag[i]))

dic_gb_ems_mean = np.asarray(dic_gb_ems_mean)
dic_gb_r_ang_mean = np.asarray(dic_gb_r_ang_mean)
dic_gb_r_ang_mag_mean = np.asarray(dic_gb_r_ang_mag_mean)

dic_gb_mori = np.asarray(dic_gb_mori)

In [320]:
# percentiles 
pc = 95

dic_gb_ems_pc95 = []
dic_gb_r_ang_pc95 = []
for i in range(len(dic_gb_ems)):
    dic_gb_ems_pc95.append(np.nanpercentile(dic_gb_ems[i],pc))
    dic_gb_r_ang_pc95.append(np.nanpercentile(dic_gb_r_ang[i],pc))

dic_gb_ems_pc95 = np.asarray(dic_gb_ems_pc95)
dic_gb_r_ang_pc95 = np.asarray(dic_gb_r_ang_pc95)

dic_gb_mori = np.asarray(dic_gb_mori)

In [75]:
# plot 2d histogram to visualise the joint distribution

fig,ax = plt.subplots()
ax.hist2d(dic_gb_mori,dic_gb_r_ang_mean,bins=100,range=[[0,65],[-5,5]],density=True,vmin=0,vmax=0.05)
ax.set_xlabel('Grain boundary misorientation / ° ')
ax.set_ylabel('In-plane rotation angle / ° ')

Text(0, 0.5, 'In-plane rotation angle / ° ')

In [126]:
import matplotlib as mpl

In [325]:
var1 = dic_gb_mori
var2 = dic_gb_ems_pc95

var1_label = 'Grain boundary misorientation / ° '
var2_label = '$E_{\mathrm{eff}}$/ - '

var1_range = [10,61]
var1_bins = 20

var2_range = [0,0.1]
var2_bins = 40

scatter_with_marg_hists(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range)


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Grain boundary misorientation / ° ', ylabel='$E_{\\mathrm{eff}}$/ - '>)

In [338]:
var1 = dic_gb_mori
var2 = dic_gb_r_ang_mag_mean

var1_label = 'Grain boundary misorientation / ° '
var2_label = 'In-plane rotation / ° '

var1_range = [10,61]
var1_bins = 20

var2_range = [0,5]
var2_bins = 40

scatter_with_marg_hists(var1,var1_label,var1_bins,var1_range,var2,var2_label,var2_bins,var2_range,99)

(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Grain boundary misorientation / ° ', ylabel='In-plane rotation / ° '>)

In [74]:
# plot 2d histogram to visualise the joint distribution

fig,ax = plt.subplots()
ax.hist2d(dic_gb_mori,dic_gb_r_ang_mean,bins=100,range=[[0,65],[-5,5]],density=True,vmin=0,vmax=0.05,weights=weights)
ax.set_xlabel('Grain boundary misorientation / ° ')
ax.set_ylabel('In-plane rotation angle / ° ')

ValueError: The weights and list don't have the same length.

In [33]:
# plot 2d histogram to visualise the joint distribution

fig,ax = plt.subplots()
ax.hist2d(dic_gb_mori,dic_gb_ems_mean,bins=100,range=[[0,65],[0,0.1]],density=True,vmin=0,vmax=10)
ax.set_xlabel('Grain boundary misorientation / ° ')
ax.set_ylabel('Max shear strain / - ')

Text(0, 0.5, 'Max shear strain / - ')

In [18]:
# plot 2d histogram to visualise the joint distribution

fig,ax = plt.subplots()
ax.hist2d(dic_gb_mori,dic_gb_r_ang_mean,bins=100,range=[[0,65],[-5,5]],density=True,vmin=0,vmax=0.05)
ax.set_xlabel('Grain boundary misorientation / ° ')
ax.set_ylabel('In-plane rotation angle / ° ')


Text(0, 0.5, 'In-plane rotation angle / ° ')

In [ ]:
# get sigma3 boundaries 
s_bounds_dic = hrdic.BoundarySet.from_ebsd_boundaries(dic_map,s_bounds)
dilation = 5

#segment strain map by boundaries type object

# sigma three boundaries
dic_map.generate_threshold_mask(s_bounds_dic.image,dilation=dilation,preview=False)
sb_mask = ~dic_map.mask

# all boundaries 
dic_map.generate_threshold_mask(dic_map.data['grain_boundaries'].image,dilation=dilation,preview=False)
gb_mask = ~dic_map.mask

# normal high angle gbs 
ha_mask = ~(gb_mask ^ sb_mask)

# grain cores 
gc_mask = ~gb_mask


# make some colours for the different areas for consistency 
colour_list = ['orangered',     # S3 boundaries
               'chartreuse',    # All boundaries
               'deepskyblue',   # HA grain boundaries
               'fuchsia'        # Grain cores 
               ]

In [ ]:
plt.figure()
plt.imshow(ems_gc,vmin=0,vmax=0.1)

In [ ]:
# sigma3 boundaries
ems_sb, e_sb, r_ang_sb = mask_and_extract(dic_map,sb_mask)

# all grain boundaries
ems_gb, e_gb, r_ang_gb = mask_and_extract(dic_map,gb_mask)

# high angle grain boundaries
ems_ha, e_ha, r_ang_ha = mask_and_extract(dic_map,ha_mask)

# grain cores
ems_gc, e_gc, r_ang_gc = mask_and_extract(dic_map,gc_mask)


# plt.imshow(r_ang_sb,vmin=-5,vmax=5,cmap='RdBu_r')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[ems_sb,ems_ha,ems_gb,ems_gc],bin_range=[0.01,0.5],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('Max shear / - ')
ax.set_ylabel('Relative frequency')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[abs(r_ang_sb),abs(r_ang_ha),abs(r_ang_gb),abs(r_ang_gc)],bin_range=[0.1,10],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('In plane rotation magnitude / degrees')
ax.set_ylabel('Relative frequency')

In [ ]:
fig,ax = plt.subplots()
fig, ax = make_log_log_histogram(fig,ax,[abs(e_sb[0,0]),abs(e_ha[0,0]),abs(e_gb[0,0]),abs(e_gc[0,0])],bin_range=[0.01,0.5],bins=100,density=True,colour_list=colour_list)
ax.set_xscale('log')
ax.set_yscale('log')
ax.xaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.5g}'.format(y)))
ax.legend(['S3GBs','HAGBs','All GBs','Grain cores'])
ax.set_xlabel('E11 / - ')
ax.set_ylabel('Relative frequency')

In [ ]:
plot = dic_map.plot_map('max_shear',vmin=0,vmax=0.1,plot_gbs=False,dilate_boundaries=True,plot_scale_bar=True)
plt.savefig('max_shear.png',dpi=500)
# plt.figure()
plot = dic_map.plot_map('max_shear',vmin=0,vmax=0.1,plot_gbs=False,dilate_boundaries=True,plot_scale_bar=True)
plot.add_grain_boundaries(kind='line',boundaries=s_bounds_dic,color='limegreen')
plt.savefig('max_shear_s3gbs.png',dpi=500)

In [ ]:
# test with a single map to make things easier
# exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
dic_map = hrdic.Map(dic_dir/ 'map_44.txt', data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth


# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)

dic_map.set_scale(pixelsize)
dic_map.set_crop(left=100,right=100,top=100,bottom=100)

ebsd_pre_frame = experiment.Frame()
ebsd_map=ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

ebsd_map.find_grains()
ebsd_map.build_neighbour_network()

In [ ]:
# exp = experiment.Experiment()

# dic_dir = Path('./DIC/')
# ebsd_pre_dir = Path('./Pre_EBSD/')
# ebsd_post_dir = Path('./Post_EBSD/')

# dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
# hfw = 20.0 # microns
# pixelwidth = 2048
# pixelsize = hfw/pixelwidth

# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)
    
# ebsd_pre_frame = experiment.Frame()
# ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.find_boundaries()
# ebsd_map.find_grains
# ebsd_map.build_neighbour_network()

In [ ]:
# ebsd_post_frame = experiment.Frame()
# ebsd.Map(ebsd_post_dir / 'map_small.cpr',increment=exp.increments[-1], frame=ebsd_post_frame)

In [ ]:
exp.increments[1].maps['ebsd']

In [ ]:
# dic_map = exp.increments[0].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

# ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

# dic_map.homog_points = [(1722, 1177),
#  (1335, 1313),
#  (948, 1745),
#  (549, 1661),
#  (245, 1680),
#  (233, 1300),
#  (579, 1158),
#  (443, 756),
#  (559, 136),
#  (83, 359),
#  (1438, 495),
#  (857, 165)]

# ebsd_map.homog_points = [(2541, 1823),
#  (2240, 1932),
#  (1940, 2266),
#  (1631, 2207),
#  (1394, 2225),
#  (1388, 1933),
#  (1659, 1820),
#  (1554, 1519),
#  (1648, 1054),
#  (1279, 1219),
#  (2323, 1319),
#  (1875, 1080)]

# dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
# dic_map.plot_map('max_shear', vmin=0,vmax=0.1,plot_scale_bar=True,plot_gbs=True)
#  # link maps
# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_crop
#     dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
#     # dic_map.plot_map(
#     #     'max_shear', vmin=0, vmax=0.10, 
#     #     plot_scale_bar=False, plot_gbs='line'
#     # )
    # plt.tight_layout()

In [ ]:
# find twin boundaries
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)
# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
twin_lines = []
for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False
    misori = grain2.ref_ori * grain1.ref_ori.conjugate
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines += b_seg.boundary_lines

In [ ]:
ebsd_map.build_neighbour_network()

In [ ]:
ebsd_map.

In [ ]:
t = ebsd_map.neighbour_network.edges.data('boundary')

In [ ]:
plot = ebsd_map.plot_ipf_map([1,0,0])
plot.add_grain_boundaries(kind='line',boundaries=twin_lines, colour='red')

In [ ]:
 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    # plt.tight_layout()

dic_map = exp.increments[-1].maps['hrdic'] 
dic_map.plot_map(
    
)